In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import gradio as gr
from PIL import Image

In [2]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(3,10,kernel_size=5)
        self.conv2=nn.Conv2d(10,20,kernel_size=5)
        self.pool1=nn.MaxPool2d(kernel_size=2, stride=2)
        self.pool2=nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1=nn.Linear(320, 50)
        self.fc2=nn.Linear(50,10)
    def forward(self,x):
        x=F.relu(self.conv1(x))
        x=self.pool1(x)
        x=F.relu(self.conv2(x))
        x=self.pool2(x)
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
        

In [6]:
model=NN()

In [7]:
model=torch.load('model/digit-recognization.pth',map_location=torch.device('cpu'))

In [8]:
model.to(device)

NN(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [9]:
from torchvision import transforms
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

In [26]:
def predict(img):
    img=img.convert('L')
    img=transform(img)
    img.to(device)
    preds=torch.argmax(model(img),1)
    return preds.item()

In [27]:
gr.Interface(fn=predict,
             inputs=gr.Image(type="pil"),
             outputs=gr.Text(),
            ).launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
